In [1]:
from perfetto.trace_processor import TraceProcessor
import altair as alt

In [16]:
tp.query("""
SELECT name FROM perfetto_tables ORDER BY name
""").as_pandas_dataframe()

,name
0,android_logs
1,args
2,counter
3,counter_track
4,cpu
5,cpu_counter_track
6,cpu_freq
7,cpu_profile_stack_sample
8,experimental_counter_dur
9,experimental_sched_upid


In [7]:
!python ../scripts/trace_scroll.py --activity=com.twitter.android/.StartActivity --trace=twitter-scroll.pftrace.gz

Stopping: com.twitter.android
Starting: Intent { act=android.intent.action.MAIN cat=[android.intent.category.LAUNCHER] cmp=com.twitter.android/.StartActivity }
Status: ok
LaunchState: COLD
Activity: com.twitter.android/com.twitter.app.main.MainActivity
TotalTime: 735
WaitTime: 737
Complete
8402
/data/misc/perfetto-traces/trace: 1 fi.... 31.8 MB/s (56892756 bytes in 1.705s)


In [9]:
!python ../scripts/trace_scroll.py --activity=com.instagram.android/.activity.MainTabActivity --trace=ig-scroll.pftrace.gz

Stopping: com.instagram.android
Starting: Intent { act=android.intent.action.MAIN cat=[android.intent.category.LAUNCHER] cmp=com.instagram.android/.activity.MainTabActivity }
Status: ok
LaunchState: COLD
Activity: com.instagram.android/com.instagram.mainactivity.MainActivity
TotalTime: 1346
WaitTime: 1347
Complete
9595
/data/misc/perfetto-traces/trace: 1 fi.... 30.9 MB/s (51161563 bytes in 1.579s)


In [10]:
tw = TraceProcessor(file_path='twitter-scroll.pftrace.gz')

In [13]:
ig = TraceProcessor(file_path='ig-scroll.pftrace.gz')

In [112]:
interval_query = """
WITH
frames AS (
  SELECT
    slice.dur dur,
    slice.ts  ts,
    slice.dur + slice.ts frame
  FROM
    slice
  INNER JOIN thread_track ON (slice.track_id = thread_track.id)
  INNER JOIN thread       ON (thread_track.utid = thread.utid)
  INNER JOIN process      ON (thread.upid       = process.upid)
  WHERE
        thread.is_main_thread = 1      
    AND process.name = '{p}'
    AND slice.name = 'Choreographer#doFrame'
),
first_frame AS (
  SELECT MIN(frames.ts) ts
  FROM frames
)
SELECT
  (frames.frame - (lag(frames.frame)
    OVER (ORDER BY frames.ts ROWS BETWEEN 1 PRECEDING AND CURRENT ROW))) / (1000 * 1000) AS interval,
  (frames.ts - first_frame.ts) / (1000 * 1000) ts
FROM frames, first_frame
"""

twdf = tw.query(interval_query.format(p='com.twitter.android')).as_pandas_dataframe()
igdf = ig.query(interval_query.format(p='com.instagram.android')).as_pandas_dataframe()

def chart_pair(df, color):
    return (alt.Chart(df).mark_point(color=color).encode(
                x='ts', y=alt.Y('interval', scale=alt.Scale(domain=(0, 200)))) | \
            alt.Chart(df).mark_bar(color=color).encode(
                x=alt.X('interval', bin=alt.Bin(step=5), scale=alt.Scale(domain=(0, 200))), y='count()'))

chart_pair(twdf, color='lightblue') & chart_pair(igdf, color='palevioletred')

alt.VConcatChart(...)

In [117]:
def frame_drop_ratio(df):
    interval = df['interval']
    return (interval > 20).sum() / len(interval)

print('TW: %f' % frame_drop_ratio(twdf))
print('IG: %f' % frame_drop_ratio(igdf))

TW: 0.341935
IG: 0.053528


### Looking at CPU usage

In [108]:
load_query = """
WITH
sched_slices AS (
  SELECT
    sched.*,
    (sched.ts / (1000 * 1000 * 100)) span
  FROM
    sched
  INNER JOIN thread       ON (sched.utid  = thread.utid)
  INNER JOIN process      ON (thread.upid = process.upid)
  WHERE
        thread.is_main_thread = 1      
    AND process.name = '{p}'
),
first_span AS (
SELECT
    MIN(sched_slices.span) as span
FROM sched_slices
)
SELECT
  CAST(SUM(sched_slices.dur) AS float) / (1000 * 1000 * 100) load,
  sched_slices.span - first_span.span span
FROM sched_slices, first_span
GROUP BY sched_slices.span
ORDER BY sched_slices.span DESC
"""

igload = ig.query(load_query.format(p='com.instagram.android')).as_pandas_dataframe()
twload = tw.query(load_query.format(p='com.twitter.android')).as_pandas_dataframe()

def sched_chart_pair(df, color):
    return alt.Chart(df).mark_point(color=color).encode(
                 x='span', y=alt.Y('load', scale=alt.Scale(domain=(0, 1)))) | \
           alt.Chart(df).mark_bar(color=color).encode(
                x=alt.X('load', bin=alt.Bin(step=0.1), scale=alt.Scale(domain=(0, 1))), y='count()')

sched_chart_pair(twload, 'lightblue') & sched_chart_pair(igload, 'palevioletred')

alt.VConcatChart(...)

In [ ]:
color=